In [5]:
import requests
import os
import json
import pandas as pd
from dotenv import load_dotenv

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [81]:
'''
Testing Connection To Foursquare API
'''


# loading in environmental variables for api keys
load_dotenv()

FOURSQUARE_API_KEY = os.getenv('FOURSQUARE_API_KEY')

# setting up headers
headers = {
    'Authorization': FOURSQUARE_API_KEY,
    'accept': 'application/json'
}

# defining latitude and longitude ranges (In this case the city chosen is Barcelona)
min_latitude = 41.346775
max_latitude = 41.462095
min_longitude = 2.109154
max_longitude = 2.220691

# defining increments 
latitude_step = 0.01
longitude_step = 0.01

# creating list to collect results while loop iterates
foursquare_results = []

latitude = min_latitude
while latitude <= max_latitude:
    longitude = min_longitude
    while longitude <= max_longitude:
        # setting parameters
        params = {
            'll': f'{latitude},{longitude}',  # latitude, longitude
            'radius': 1000,
            'query': 'restaurant',
            'limit': 20                  
        }
        # request to foursquare api
        foursquare_response = requests.get('https://api.foursquare.com/v3/places/search', headers=headers, params=params)

        # check if request was successful
        if foursquare_response.status_code == 200:
            # parsing json response
            foursquare_data = foursquare_response.json()
            if foursquare_data['results'] and len(foursquare_data['results']) > 0:
                # append results to foursquare_results
                foursquare_results.extend(foursquare_data['results']) # using .extend due to multiple items
            else:
                print(f"No businesses found for latitude: {latitude}, longitude: {longitude}")
        else: 
            print(f"Error {foursquare_response.status_code} for {latitude}, {longitude}")
    
        # increment longitude
        longitude += longitude_step
    
    # increment latitude
    latitude += latitude_step

# Creating dataframe 
if foursquare_results:
    foursquare_results_df = pd.json_normalize(foursquare_results)
    print(foursquare_results_df)
else:
    print('No results')

No businesses found for latitude: 41.346775, longitude: 2.169153999999999
No businesses found for latitude: 41.346775, longitude: 2.1791539999999987
No businesses found for latitude: 41.346775, longitude: 2.1891539999999985
No businesses found for latitude: 41.346775, longitude: 2.1991539999999983
No businesses found for latitude: 41.346775, longitude: 2.209153999999998
No businesses found for latitude: 41.346775, longitude: 2.219153999999998
No businesses found for latitude: 41.356775, longitude: 2.1891539999999985
No businesses found for latitude: 41.356775, longitude: 2.1991539999999983
No businesses found for latitude: 41.356775, longitude: 2.209153999999998
No businesses found for latitude: 41.356775, longitude: 2.219153999999998
No businesses found for latitude: 41.366775, longitude: 2.209153999999998
No businesses found for latitude: 41.366775, longitude: 2.219153999999998
No businesses found for latitude: 41.376774999999995, longitude: 2.209153999999998
No businesses found for 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [82]:
foursquare_results_df

,fsq_id,categories,chains,closed_bucket,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,...,location.postcode,location.region,related_places.parent.fsq_id,related_places.parent.categories,related_places.parent.name,location.cross_street,geocodes.drop_off.latitude,geocodes.drop_off.longitude,location.address_extended,related_places.children
0,4adcda55f964a5203e4321e3,"[{'id': 13302, 'name': 'Mediterranean Restaura...",[],VeryLikelyOpen,95,/v3/places/4adcda55f964a5203e4321e3,Azimut Restaurant,Europe/Madrid,41.346118,2.108411,...,08907,Catalunya,4b83b43bf964a520960d31e3,"[{'id': 19014, 'name': 'Hotel', 'short_name': ...",Hotel Hesperia Tower,NaN,NaN,NaN,NaN,NaN
1,4bf521f6706e20a1370daa98,"[{'id': 13311, 'name': 'Molecular Gastronomy R...",[],LikelyOpen,90,/v3/places/4bf521f6706e20a1370daa98,Evo,Europe/Madrid,41.346035,2.108595,...,08902,Catalunya,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
2,4adcda55f964a5203d4321e3,"[{'id': 13065, 'name': 'Restaurant', 'short_na...",[],LikelyOpen,95,/v3/places/4adcda55f964a5203d4321e3,Bouquet,Europe/Madrid,41.346118,2.108411,...,08907,Catalunya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4ceb9649f8653704a37fc3c4,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",[],VeryLikelyOpen,191,/v3/places/4ceb9649f8653704a37fc3c4,365.Cafè,Europe/Madrid,41.348221,2.107895,...,08907,Catalunya,NaN,NaN,NaN,NaN,41.348173,2.108082,NaN,NaN
4,610b02aed3070f630819d199,"[{'id': 13345, 'name': 'Spanish Restaurant', '...",[],VeryLikelyOpen,187,/v3/places/610b02aed3070f630819d199,Bodega Ermita,Europe/Madrid,41.348456,2.108315,...,08907,Catalunya,NaN,NaN,NaN,,41.348505,2.108303,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,58149a5238fae85660c5b7b3,"[{'id': 13055, 'name': 'Fried Chicken Joint', ...",[{'id': 'ab4beec0-d68a-012e-5619-003048cad9da'...,LikelyOpen,716,/v3/places/58149a5238fae85660c5b7b3,KFC,Europe/Madrid,41.455749,2.227650,...,08917,Catalunya,4de8c178d1648c979639a468,"[{'id': 17114, 'name': 'Shopping Mall', 'short...",Centre Comercial Montigalà,,NaN,NaN,NaN,NaN
1942,4d3c244b3ec9a35d46f04581,"[{'id': 13064, 'name': 'Pizzeria', 'short_name...",[],LikelyOpen,1003,/v3/places/4d3c244b3ec9a35d46f04581,La Tagliatella,Europe/Madrid,41.452346,2.208726,...,08921,Catalunya,NaN,NaN,NaN,,41.452464,2.208548,Bajo,NaN
1943,4bb49bfad1b0c9b622139daf,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],LikelyOpen,956,/v3/places/4bb49bfad1b0c9b622139daf,Bar Olimaja,Europe/Madrid,41.449973,2.212120,...,08922,Catalunya,NaN,NaN,NaN,Sant Silvestre,41.449852,2.212100,NaN,NaN
1944,4dea2754d4c00071b81a9066,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],LikelyOpen,967,/v3/places/4dea2754d4c00071b81a9066,El Racó del Cafè,Europe/Madrid,41.451092,2.210351,...,NaN,Catalunya,NaN,NaN,NaN,,41.450989,2.210333,NaN,NaN


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [75]:
'''
Testing Connection To Yelp API
'''

# getting yelp api key
YELP_API_KEY = os.getenv('YELP_API_KEY')

# setting parameters
params = {
    'latitude': '41.405520',
    'longitude': '2.16225', 
    'limit': 5,
    'radius': 1000
}

# setting headers
headers = {
    'Authorization': f'Bearer {YELP_API_KEY}'
}

# defining latitude and longitude ranges (In this case the city chosen is Barcelona)
min_latitude = 41.346775
max_latitude = 41.462095
min_longitude = 2.109154
max_longitude = 2.220691

# defining increments 
latitude_step = 0.01
longitude_step = 0.01

# list to collect results while loop iterates
yelp_results = []

# looping through ranges of latitude and longitude
latitude = min_latitude
while latitude <= max_latitude:
    longitude = min_longitude
    while longitude <= max_longitude:
        # setting parameters for api request
        params = {
            'latitude': latitude,
            'longitude': longitude, 
            'limit': 20,
            'radius': 1000
        }
    
        # making request to api
        yelp_response = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params=params)

        # check if request was successful
        if yelp_response.status_code == 200:
            yelp_data = yelp_response.json()

            if 'businesses' in yelp_data and len(yelp_data['businesses']) > 0:
                # appending businesses to yelp_results list
                yelp_results.extend(yelp_data['businesses']) # using .extend due to multiple items
            else:
                print(f"No businesses found for latitude: {latitude}, longitude: {longitude}")
        else: 
            print(f"Error {yelp_response.status_code} for {latitude}, {longitude}")
    
        # increment longitude
        longitude += longitude_step
    
    # increment latitude
    latitude += latitude_step

# Creating dataframe 
if yelp_results:
    yelp_results_df = pd.json_normalize(yelp_results)
    print(yelp_results_df)
else:
    print('No results')

No businesses found for latitude: 41.346775, longitude: 2.159153999999999
No businesses found for latitude: 41.346775, longitude: 2.169153999999999
No businesses found for latitude: 41.346775, longitude: 2.1791539999999987
No businesses found for latitude: 41.346775, longitude: 2.1891539999999985
No businesses found for latitude: 41.346775, longitude: 2.1991539999999983
No businesses found for latitude: 41.346775, longitude: 2.209153999999998
No businesses found for latitude: 41.346775, longitude: 2.219153999999998
No businesses found for latitude: 41.356775, longitude: 2.1891539999999985
No businesses found for latitude: 41.356775, longitude: 2.1991539999999983
No businesses found for latitude: 41.356775, longitude: 2.209153999999998
No businesses found for latitude: 41.356775, longitude: 2.219153999999998
No businesses found for latitude: 41.366775, longitude: 2.209153999999998
No businesses found for latitude: 41.366775, longitude: 2.219153999999998
No businesses found for latitude:

In [77]:
yelp_results_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,location.city,location.zip_code,location.country,location.state,location.display_address,attributes.business_temp_closed,attributes.menu_url,attributes.open24_hours,attributes.waitlist_reservation,price
0,peGMNEAPUxOR4Trg15eVdA,le-bouquet-l-hospitalet-de-llobregat,Le Bouquet,,False,https://www.yelp.com/biz/le-bouquet-l-hospital...,3,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",4.3,[],...,L'Hospitalet de Llobregat,08907,ES,B,"[Hotel Hesperia Tower, Gran Vía, 144, 08907 L'...",None,None,None,None,NaN
1,0_UULyMycCwHIqK0avhvxg,el-xiringuito-de-bellvitge-l-hospitalet-de-llo...,El Xiringuito de Bellvitge,https://s3-media2.fl.yelpcdn.com/bphoto/Mfx8yI...,False,https://www.yelp.com/biz/el-xiringuito-de-bell...,3,"[{'alias': 'tapas', 'title': 'Tapas Bars'}]",4.7,[],...,L'Hospitalet de Llobregat,08901,ES,B,"[Rambla de la Marina, s/n, 08901 L'Hospitalet ...",None,None,None,None,€
2,C65Z0hoJWndFHilQx6RxXw,bar-can-galan-hospitalet-de-llobregat,Bar Can Galan,https://s3-media3.fl.yelpcdn.com/bphoto/6y3-6y...,False,https://www.yelp.com/biz/bar-can-galan-hospita...,5,"[{'alias': 'catalan', 'title': 'Catalan'}]",3.6,[],...,Hospitalet de Llobregat,08907,ES,B,"[Avinguda America, 37, 08907 Hospitalet de Llo...",None,None,None,None,€
3,Ldwla3RbG0qq4UW4yVZm6w,la-marina-hospitalet-de-llobregat,La Marina,https://s3-media1.fl.yelpcdn.com/bphoto/0HWrnp...,False,https://www.yelp.com/biz/la-marina-hospitalet-...,2,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",4.0,[],...,Hospitalet de Llobregat,08907,ES,B,"[Carrer de la Feixa Llarga, 47, 08907 Hospital...",None,None,None,None,NaN
4,60GlRETCW6AXCS6d4gWQUA,da-fu-hau-hospitalet-de-llobregat,Da Fu Hau,https://s3-media4.fl.yelpcdn.com/bphoto/btmdbc...,False,https://www.yelp.com/biz/da-fu-hau-hospitalet-...,2,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,[],...,Hospitalet De Llobregat,08907,ES,B,"[Rambla Marina, 353, 08907 Hospitalet De Llobr...",None,None,None,None,€
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,q76Gm87sn4-tM0LDcA72mA,can-sardà-cerdanyola-del-vallès-3,Can Sardà,https://s3-media2.fl.yelpcdn.com/bphoto/MR8IDB...,False,https://www.yelp.com/biz/can-sard%C3%A0-cerdan...,2,"[{'alias': 'catalan', 'title': 'Catalan'}]",4.0,[],...,Cerdanyola del Vallès,08290,ES,B,"[Carretera de Horta, Km 4, 08290 Cerdanyola de...",None,None,None,None,€€€
1957,nmkQ1DebRdHrl1R3A629Wg,can-valldaura-cerdanyola-del-valles,Can Valldaura,https://s3-media3.fl.yelpcdn.com/bphoto/8g4Tgs...,False,https://www.yelp.com/biz/can-valldaura-cerdany...,3,"[{'alias': 'brasseries', 'title': 'Brasseries'}]",3.0,[],...,Cerdanyola del Valles,08290,ES,B,"[BV-1415 Km. 7,08, 08290 Cerdanyola del Valles...",None,None,None,None,€€
1958,jjtoB00GRSXRgdpW9PnxuQ,masía-can-borrell-barcelona,Masía Can Borrell,https://s3-media2.fl.yelpcdn.com/bphoto/MWjqu8...,False,https://www.yelp.com/biz/mas%C3%ADa-can-borrel...,5,"[{'alias': 'spanish', 'title': 'Spanish'}]",3.8,[],...,Barcelona,08290,ES,B,"[Carretera d'Horta a Cerdanyola BV-1415, km. 3...",None,None,None,None,€€
1959,q76Gm87sn4-tM0LDcA72mA,can-sardà-cerdanyola-del-vallès-3,Can Sardà,https://s3-media2.fl.yelpcdn.com/bphoto/MR8IDB...,False,https://www.yelp.com/biz/can-sard%C3%A0-cerdan...,2,"[{'alias': 'catalan', 'title': 'Catalan'}]",4.0,[],...,Cerdanyola del Vallès,08290,ES,B,"[Carretera de Horta, Km 4, 08290 Cerdanyola de...",None,None,None,None,€€€


Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp's data is more comprehensive, as it gives me ratings and amount of reviews per location, which is useful in what I am going to use for my model (seeing if there is a relationship between amount of bikes at a station and amount of reviews for POIs)

Get the top 10 restaurants according to their rating

In [78]:
yelp_results_df.sort_values(by='rating', ascending=False).head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,location.city,location.zip_code,location.country,location.state,location.display_address,attributes.business_temp_closed,attributes.menu_url,attributes.open24_hours,attributes.waitlist_reservation,price
104,djRrSqGWdMpYcsEwcEaLGw,restaurantoc-barcelona,RestauranToc,https://s3-media1.fl.yelpcdn.com/bphoto/SfqLkW...,False,https://www.yelp.com/biz/restaurantoc-barcelon...,1,"[{'alias': 'mediterranean', 'title': 'Mediterr...",5.0,[],...,Barcelona,08038,ES,B,"[C/60 N9, zona franca, 08038 Barcelona, Spain]",None,None,None,None,€€€€
1478,mM9eXP63UmkGEujt3Ks2VA,mont-escape-barcelona,Mont Escape,https://s3-media2.fl.yelpcdn.com/bphoto/IuzDN7...,False,https://www.yelp.com/biz/mont-escape-barcelona...,1,"[{'alias': 'tours', 'title': 'Tours'}, {'alias...",5.0,[],...,Barcelona,,ES,B,"[Barcelona, Spain]",None,NaN,NaN,None,NaN
1336,JzFoR6byU727TcNh-ULSvQ,el-mirador-del-carmelo-barcelona,El Mirador del Carmelo,,False,https://www.yelp.com/biz/el-mirador-del-carmel...,2,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",5.0,[],...,Barcelona,08024,ES,B,"[Calle Pasteur 2, 08024 Barcelona, Spain]",None,None,None,None,NaN
1476,frqxodS5BD9ME4d-F6Yxdw,l-orangerie-barcelona,L'Orangerie,https://s3-media2.fl.yelpcdn.com/bphoto/JT5c4r...,False,https://www.yelp.com/biz/l-orangerie-barcelona...,1,"[{'alias': 'catalan', 'title': 'Catalan'}, {'a...",5.0,[],...,Barcelona,08035,ES,B,"[Carretera Vallvidrera Tibidabo, 83, 08035 Bar...",None,None,None,None,€€€
1473,mM9eXP63UmkGEujt3Ks2VA,mont-escape-barcelona,Mont Escape,https://s3-media2.fl.yelpcdn.com/bphoto/IuzDN7...,False,https://www.yelp.com/biz/mont-escape-barcelona...,1,"[{'alias': 'tours', 'title': 'Tours'}, {'alias...",5.0,[],...,Barcelona,,ES,B,"[Barcelona, Spain]",None,NaN,NaN,None,NaN
1724,z4YAxNuaqjJSv5Z7d_9Q8A,parc-central-barcelona,Parc Central,https://s3-media3.fl.yelpcdn.com/bphoto/q4tORY...,False,https://www.yelp.com/biz/parc-central-barcelon...,1,"[{'alias': 'tapas', 'title': 'Tapas Bars'}]",5.0,[],...,Barcelona,08042,ES,B,"[Passeig Fabra y Puig, 468, 08042 Barcelona, S...",None,None,None,None,NaN
1723,SkKPvSXl4ITIKy86qYsYew,paparazzi-bar-barcelona,Paparazzi Bar,https://s3-media3.fl.yelpcdn.com/bphoto/WdydLw...,False,https://www.yelp.com/biz/paparazzi-bar-barcelo...,1,"[{'alias': 'andalusian', 'title': 'Andalusian'...",5.0,[],...,Barcelona,08042,ES,B,"[Via Júlia, 201, 08042 Barcelona, Spain]",None,http://www.paparazzi-bar.net/carta,None,None,NaN
1341,v5z1XE3kh0F1WpVjPeOj9Q,el-jardí-de-can-toda-barcelona,El Jardí de Can Toda,https://s3-media4.fl.yelpcdn.com/bphoto/0WF9BG...,False,https://www.yelp.com/biz/el-jard%C3%AD-de-can-...,1,"[{'alias': 'mediterranean', 'title': 'Mediterr...",5.0,[],...,Barcelona,08024,ES,B,"[Carrer de Ramiro de Maetzu, 31, 08024 Barcelo...",None,None,None,None,€
1720,a__rSnUHEyM5X8ngcRIlLA,vinomio-barcelona,Vinomio,https://s3-media4.fl.yelpcdn.com/bphoto/MPvypm...,False,https://www.yelp.com/biz/vinomio-barcelona?adj...,1,"[{'alias': 'mediterranean', 'title': 'Mediterr...",5.0,[],...,Barcelona,08032,ES,B,"[Carrer Tajo, 19, 08032 Barcelona, Spain]",None,http://www.vinomiohorta.com/Men%C3%BAs/,None,None,€€€
1470,UqzMynpMcpRFCdQ-YdVyfg,can-cortes-planes,Can Cortes,https://s3-media2.fl.yelpcdn.com/bphoto/zK6_6H...,False,https://www.yelp.com/biz/can-cortes-planes?adj...,1,"[{'alias': 'catalan', 'title': 'Catalan'}]",5.0,[],...,Planes,08196,ES,B,"[Carretera de l'Arrabassada, km 6,5, Avinguda ...",None,http://www.cancortes.com/menu.php,None,None,€€


In [83]:
yelp_results_df.to_csv('yelp_results_barcelona', index=False)